In [ ]:
#installations for necessary packages   
"""
!pip install cdsapi
!pip install shutil
!pip install zipfile
!pip install xarray(complete)
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install cartopy """

In [ ]:
import cdsapi
import shutil
import zipfile
import os 
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import warnings
import pandas as pd
import math
import numpy as np
from scipy.stats import pearsonr

Installing CDS API

If you are not familiar with the Copernicus Climate Data Store API, please view the following step by step guide for installation: 
Windows: https://confluence.ecmwf.int/display/CKB/How+to+install+and+use+CDS+API+on+Windows
MacOS: https://confluence.ecmwf.int/display/CKB/How+to+install+and+use+CDS+API+on+macOS
Linux: https://cds.climate.copernicus.eu/api-how-to



In [ ]:
#must set up Copernicus Climate Data Store API before running code
c = cdsapi.Client()          
months = ['01', '02','03', '04', '05', '06', '07', '08', '09', '10', '11', '12']  #months of the year
destination_dir = 'SSTdata'   #specify directory for where nc files will go

for month in months:
    os.makedirs(destination_dir, exist_ok = True)    #make directory
    
    c.retrieve(
        'satellite-sea-surface-temperature',
        {
            'version': '2_1',
            'variable': 'all',
            'format': 'zip',
            'processinglevel': 'level_4',
            'year': [
                '1997', '1998', '1999',
                '2000', '2001', '2002',
                '2003', '2004', '2005',
                '2006', '2007', '2008',
                '2009', '2010',
            ],
            'month': month,
            'day': [
                '01', '04', '07',
                '10', '13', '16',
                '19', '22', '25',
                '28',
            ],
            'sensor_on_satellite': 'combined_product',
        },
        f"SSTmonth{month}.zip")   #download global SST for every three days while looping through each month. 
    
    shutil.move(f"SSTmonth{month}.zip", destination_dir)       
    with zipfile.ZipFile(f"{destination_dir}/SSTmonth{month}.zip", 'r') as zip_ref:
        zip_ref.extractall(destination_dir)     #move files to directory and extract zip files into directory
        
    if os.path.isfile(f"{destination_dir}/SSTmonth{month}.zip"):
        os.remove(f"{destination_dir}/SSTmonth{month}.zip")      #delete zip files
        
    print(f"Download for month {month} complete")
print('All downloads complete')

In [ ]:
#This code will download all the wind data for later processing. Already selected for the study area

years = [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 
        2007, 2008, 2009, 2010]     #For this request, must loop through years.

destination_dir = 'WINDdata'   #specify directory for where nc files will go

c = cdsapi.Client()
for year in years:
    c.retrieve(
        'reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': [
                '10m_u_component_of_wind','10m_v_component_of_wind'
            ],
            'year': year,
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '04', '07',
                '10', '13', '16',
                '19', '22', '25',
                '28',
            ],
            'time': [
                '00:00', '05:00', '10:00',
                '15:00', '20:00',
            ],
            'area': [
                31, -91, 18,
                -80,
            ],
        },
        f"WINDyear{year}.nc")
    
    shutil.move(f"WINDyear{year}.nc", destination_dir)  #these files are not zip, so no need to delete after like in previous code       
        
    print(f"Download for year {year} complete")
print('All downloads complete')

In [ ]:
ds = xr.open_mfdataset('SSTdata/*.nc', parallel=True) #access all SST files as one variable

In [ ]:
#reducing global data down to study area
gomslice = ds.analysed_sst.sel(
    lat=slice(18,31),
    lon=slice(-91,-80))
gomslice.load().to_netcdf('SSTdata/GoMSSTdata.nc')   #saving as new file. At this point, the previous nc files can be deleted if desired.

In [ ]:
ds = xr.open_dataset('SSTdata/GoMSSTdata.nc')

In [ ]:
#Converting from Kelvin to degrees Celsius and calculating SST anomaly

ds['analysed_sst'] = ds['analysed_sst'] - 273.15       #Kelvin to Celsius
monthly_mean_sst = ds.analysed_sst.groupby('time.month').mean(dim='time')   #Making new dataset with only monthly mean SST    
ssta = ds.analysed_sst.groupby('time.month')-monthly_mean_sst               #Subtracting monthly mean dataset from ds to obtain anomaly
ds = ssta.to_dataset()
ds = ds.rename({'analysed_sst':'ssta'})
ds

In [ ]:
#Plotting to observe study area
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ds['ssta'][0,:,:].plot(ax=ax)
ax.coastlines()
plt.show()


In [ ]:
#converting data time values to weekly for comparison with Weisberg et al. 2014
end_date = '2010-12-31'
weekly_dates = pd.date_range(start=ds.time[0].values, end=end_date, freq='W') #generating weekly dates from the first date in data to '2010-12-31
ds_weekly = ds.interp(time=weekly_dates).copy()
ds_weekly

Generating Trackline 26

In [ ]:
# Trackline code originally referenced from exercise provided by instructor
# This code is generated by ChatGPT 3.5
# Define the start and end points
start_point = (-85.79403277044844, 28.44874852567387)
end_point = (-83.61900543369907, 23.84111494192613)

# Calculate the distance between the two points using Vincenty's formulae
def vincenty_distance(lat1, lon1, lat2, lon2):
    earth_radius = 6371  # Earth's radius in kilometers
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi/2) * math.sin(delta_phi/2) + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2) * math.sin(delta_lambda/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = earth_radius * c

    return distance

total_distance = vincenty_distance(start_point[1], start_point[0], end_point[1], end_point[0])

# Calculate intermediate points and store them in a NumPy array
num_points = 100
intermediate_points = np.zeros((num_points, 2))  # Initialize a NumPy array to store the points

for i in range(num_points):
    ratio = (i + 1) / (num_points + 1)
    intermediate_lat = start_point[0] + (end_point[0] - start_point[0]) * ratio
    intermediate_lon = start_point[1] + (end_point[1] - start_point[1]) * ratio
    intermediate_points[i] = [intermediate_lat, intermediate_lon]

print('intermediate_points has shape', intermediate_points.shape)
print('Start point',intermediate_points[0])
print('End point',intermediate_points[-1])

In [ ]:
# Make coordinate of 100 points aling trackine 26 consistant with SLA data
points= intermediate_points.copy()
points[:, [0, 1]] = points[:, [1, 0]] # Swap the columns
print(points.shape)
print('latitudes, longitudes')
print(points[0])
print(points[-1])

In [ ]:
#Revealing trackline 26 on a plot
plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ds['ssta'][0,:,:].plot(ax=ax)
ax.plot(intermediate_points[:,0],intermediate_points[:,1], color='black', linewidth=2, zorder = 15)
ax.text(0.535, 0.6, 'Track 026', transform=ax.transAxes, 
        color='black', fontsize=10,  ha='center',  va='center',  rotation=300, zorder = 15)
ax.coastlines()
plt.show()

Extracting Data Along Trackline 26

In [ ]:
time = ds_weekly.time
latitude = points[:, 0]
longitude = points[:, 1]

# Define the size of your data
size = (len(time), len(latitude))

# Create NaN-filled array for sea surface temperature
sst = np.full(size, np.nan) 

# Create a DataArray with sst values
sst_data_array = xr.DataArray(
    sst,
    dims=('time', 'latitude'),
    coords={'time': time, 'latitude': latitude},
    name='sst'
)

# Create a Dataset with sst variable
ds_TL26 = xr.Dataset({'sst': sst_data_array})

# Add longitude coordinate
ds_TL26.coords['longitude'] = ('latitude', longitude)

# Add metadata to the DataArray
ds_TL26.sst.attrs['Long Name'] = 'sea_surface_temperature'
ds_TL26.sst.attrs['Short Name'] = 'sst'
ds_TL26.sst.attrs['Units'] = 'Celsius'
ds_TL26.sst.attrs['Description'] = "Sea surface temperature along trackline 26 "

In [ ]:
# You can create a grid of coordinates for which you want to interpolate values
time_coords = ds_TL26.time
latitude_coords = ds_TL26.latitude
longitude_coords = ds_TL26.longitude

# Use meshgrid to create a full grid of (time, latitude, longitude)
time_grid, lat_grid, lon_grid = xr.broadcast(time_coords, latitude_coords, longitude_coords)

# Interpolate using the interp method over the entire grid
interpolated_ds = ds.analysed_sst.interp(time=time_grid, lat=lat_grid, lon=lon_grid, method='linear')

# Now interpolated_ds contains the interpolated values for the entire grid
# Assuming that the original shape of ds_TL26.sla is compatible, you can assign the values directly
ds_TL26['sst'] = interpolated_ds

ds_TL26.sst

In [ ]:
# Create a grid of time and latitude values for contour plot
lat_grid, time_grid = np.meshgrid(ds_TL26.latitude,ds_TL26.time,)

# Create a Figure and Axes object
fig, ax = plt.subplots(figsize=(14, 6))

# Create the contour plot
contour = ax.contourf(time_grid,lat_grid, ds_TL26.sst, cmap='jet')

# Add colorbar
cbar = fig.colorbar(contour, ax=ax, orientation='horizontal',shrink=0.6)
cbar.set_label('Sea Surface Temperature Anomaly (C)')

# Set labels and title
ax.set_xlabel('Time (year)')
ax.set_ylabel('Latitude')

# Format x-axis tick labels to display only last two digits of the year
import matplotlib.dates as mdates

# Set x-axis major locator and formatter to show ticks and labels for every year
years = mdates.YearLocator()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%y'))  # Use '%y' to display only the last two digits of the year

K. Brevis cell count data sourced from the National Centers for Environmental Infomration (NCEI):
https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.nodc:0120767

In [ ]:
#After downloading KBrevis CSV data from above and renaming to 'kbrevis.csv'

df = pd.read_csv('kbrevis.csv', low_memory=False)
columns = ['STATE_ID', 'LATITUDE', 'LONGITUDE', 'SAMPLE_DATE', 'CELLCOUNT']
df=df[columns]
#selecting all data collected in Florida

fl_mask = (df['STATE_ID'] == 'FL')
df = df[fl_mask]
#selecting all data collected from Naples to Tampa Bay based on latitude

west_coast_mask = ((df['LATITUDE'] >= 26) & (df['LATITUDE'] <= 28))
df = df[west_coast_mask]
#converting sample date to datetime format and setting as index

df['SAMPLE_DATE'] = pd.to_datetime(df['SAMPLE_DATE'])
df.set_index('SAMPLE_DATE', inplace=True)
df.sort_index(inplace=True)
#matching start and end dates with sst data

start_date = '1997-01-01'
end_date = '2011-01-01'
df = df[start_date:end_date]

Kbrevis = df['CELLCOUNT'].resample('W').mean()
Kbrevis 

In [ ]:
#Plotting SST Anomaly contour with Kbrevis data
fig, ax = plt.subplots(2, 1, sharex=True, figsize=(14, 8))
ax[0].plot(Kbrevis, label = 'West Florida K. brevis')
ax[0].set_yscale('log')
ax[0].set_ylim(1e5, None)
ax[0].set_ylabel('Cell Count (Cells/L)')

years = mdates.YearLocator()
ax[0].xaxis.set_major_locator(years)
ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

lat_grid, time_grid = np.meshgrid(ds_TL26.latitude,ds_TL26.time)

# Create the contour plot
contour = ax[1].contourf(time_grid,lat_grid, ds_TL26.sst, cmap='jet')

# Add colorbar
cbar = fig.colorbar(contour, ax=ax, orientation='horizontal',shrink=0.6)
cbar.set_label('Sea Surface Temperature Anomaly (C)')

# Set labels and title
ax[1].set_xlabel('Time (year)')
ax[1].set_ylabel('Latitude')

In [ ]:
#Separating SST anomaly data along trackline into one-latitude chunks to find region with best correlation to K. brevis
target_lats = [24, 25, 26, 27, 28]
for upper_lat in target_lats:
    
    #obtains index of the value closest to the target latitude
    idx_target = np.argmin(np.abs(ds_TL26.latitude.values - upper_lat))
    
    #obtains index of the value closest to one minus the target latitude to obtain ~1-latitude ranges
    idx_start = np.argmin(np.abs(ds_TL26.latitude.values - upper_lat -1))
    
    #slicing by the found indices
    ds_TL26_LowLat = ds_TL26.isel(latitude=slice(idx_start, idx_target)).copy()  
    
    #plotting
    fig, ax= plt.subplots(figsize=(8, 2))
    lat_grid, time_grid = np.meshgrid(ds_TL26_LowLat.latitude,ds_TL26_LowLat.time,)
    contour = ax.contourf(time_grid, lat_grid, ds_TL26_LowLat.sst, cmap='jet')
    cbar = plt.colorbar(contour, ax=ax)
    cbar.set_label('SST Anomaly (C)')
    ax.set_xlabel('Time')
    ax.set_ylabel('Latitude')




In [ ]:
#Alternative visualization with just SST anomaly below zero
fig, ax = plt.subplots(6, 1, figsize = (12,15))
ax[0].plot(Kbrevis, label = 'West Florida K. brevis', color='red')
ax[0].set_yscale('log')
ax[0].set_ylim(1e5, None)
ax[0].set_ylabel('Cell Count (Cells/L)')
ax[0].legend()

years = mdates.YearLocator()
ax[0].xaxis.set_major_locator(years)
ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

target_lats = [24, 25, 26, 27, 28]
for i, upper_lat in enumerate (target_lats):
    idx_target = np.argmin(np.abs(ds_TL26.latitude.values - upper_lat))
    idx_start = np.argmin(np.abs(ds_TL26.latitude.values - upper_lat -1))
    ds_TL26_LowLat = ds_TL26.isel(latitude=slice(idx_start, idx_target)).copy()
    
    minlat = ds_TL26_LowLat['latitude'].min().values
    maxlat = ds_TL26_LowLat['latitude'].max().values    #obtaining min and max lat values for labeling legend
    
    ds_TL26_LowLat_mean = ds_TL26_LowLat.sst.mean(dim='latitude').copy()    #collapses latitude dimension
    ds_TL26_LowLat_mean.to_dataset()
    #selecting all cold values (SST anomaly less than 0, or else SST anomaly is 0)
    ds_TL26_LowLat_mean = ds_TL26_LowLat_mean.where(ds_TL26_LowLat_mean < 0, 0) 
    
    #Plotting
    ax[i+1].plot(ds_TL26_LowLat.time, ds_TL26_LowLat_mean, label = 
                 f"Latitude {round(float(minlat), 2)} - {round(float(maxlat), 2)}")  #labeling each subplot with min and max lat
    ax[i+1].legend()
    ax[i+1].xaxis.set_major_locator(years)
    ax[i+1].xaxis.set_major_formatter(mdates.DateFormatter('%Y')) #setting yearly x-axis
    

In [ ]:
#loop for calculating correlations by latitude chunk, but values are very low
Kbrevis = Kbrevis.fillna(0)              #filling NaN values with 0
target_lats = [24, 25, 26, 27, 28]
for i, upper_lat in enumerate (target_lats):
    idx_target = np.argmin(np.abs(ds_TL26.latitude.values - upper_lat))           
    idx_start = np.argmin(np.abs(ds_TL26.latitude.values - upper_lat -1))
    ds_TL26_LowLat = ds_TL26.isel(latitude=slice(idx_start, idx_target)).copy()
    ds_TL26_LowLat = ds_TL26_LowLat.sel(time=slice('1997-01-01', '2009-12-31'))
    
    Kbrevis = Kbrevis.loc['1997-01-01':'2009-12-31'] 
                                        
    minlat = ds_TL26_LowLat['latitude'].min().values
    maxlat = ds_TL26_LowLat['latitude'].max().values
    
    ds_TL26_LowLat_mean = ds_TL26_LowLat.sst.mean(dim='latitude').copy()
    ds_TL26_LowLat_mean.to_dataset()
    #selecting all cold values (SST anomaly less than 0, or else SST anomaly = 0)
    ds_TL26_LowLat_mean = ds_TL26_LowLat_mean.where(ds_TL26_LowLat_mean < 0, 0)   

    corr, p_value = pearsonr(ds_TL26_LowLat_mean, Kbrevis)
    
    print(f"Correlation coefficient for latitudes {minlat} - {maxlat}: {corr}")
    

In [ ]:
windds = xr.open_mfdataset('WINDdata/*.nc', parallel = True)  #opening all wind nc files 

In [ ]:
#Resampling wind data to monthly mean 
new_wind = windds.sel(time = slice('1997-01-01', '2010-12-26'))    
new_wind = windds.resample(time='1M').mean()

Upwelling Index equations derived from:
Jayaram, Chiranjivi, and Felix Jose. “Relative Dominance of Wind Stress Curl and Ekman Transport on Coastal Upwelling During Summer Monsoon in the Southeastern Arabian Sea.” Continental Shelf Research, vol. 244, Elsevier BV, July 2022, p. 104782. Crossref, https://doi.org/10.1016/j.csr.2022.104782.

In [ ]:
#Calculating upwelling index
P = 1.225 #kg/m^3  air density
Pw = 1024 #kg/m^3  seawater density
Cd = 2.6 * 10**-3 #drag coefficient for moderate winds
f = 0.622 * 10**-4 #s^-1, coriolis parameter
def upwelling_index (u, v):
    W = np.sqrt(u**2 + v**2)                                          #calculating net wind
    thetaW = (180-155.5)+(np.degrees(np.arctan(W)))-90                #angle of the wind with respect to the Florida shoreline
    Ushorecomp = [(u*np.cos(thetaW-180))+(v*np.sin(thetaW-180))]      #component of wind parallel to the shoreline
    t = P*Cd*np.abs(W)*Ushorecomp                                     #along shore compnent of wind shear stress
    
    EkmanTransport = t/(Pw*f)                                         #Ekman Transport equation (m^3 per second)
    
    return EkmanTransport
    
#Using upwelling index function to fill new variable in wind dataset with the ekman transport values
EkmTrans_data = upwelling_index(new_wind['u10'].values, new_wind['v10'].values)       # (u, v)  
EkmTrans_data = np.squeeze(EkmTrans_data, axis = 0)
EkmTrans_data = xr.DataArray(EkmTrans_data, dims=['time','latitude', 'longitude'], name='EkmTrans')

new_wind['EkmTrans'] = EkmTrans_data  #new variable 'EkmTrans' filled with calculated value
    

In [ ]:
#making a copy of the SST anomaly dataset to resampled to monthly mean to plot together with the wind data
dsmonthly = ds.resample(time = '1M').mean()

In [ ]:
#Plotting maps of SST anomaly overlayed with wind direction quivers
dates = ['2003-01-31','2004-01-31','2005-01-31', '2006-01-31', '2007-01-31', '2008-01-31','2009-01-31', '2010-01-31', '2011-01-31']

longitude = new_wind['longitude'].values
latitude = new_wind['latitude'].values         #lon, lat for input into np.meshgrid

lon, lat = np.meshgrid(longitude, latitude)

extent = [-90, -79, 23.5, 31]                 #map extent

fig, axes = plt.subplots(nrows = 3, ncols = 3 , figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})

for i, date in enumerate(dates):
    ax = axes.flat[i]
    #plotting monthly SST anomaly, reduce 'shrink' if colorbars are too large
    dsmonthly.sel(time=date, method='nearest')['ssta'].plot(ax=ax, cbar_kwargs={'shrink': 1, 'label': 'SST Anomaly'})
    
    #setup for quiver function 
    u10 = new_wind['u10'].sel(time=date, method='nearest').values
    v10 = new_wind['v10'].sel(time=date, method='nearest').values
    
    stride = 5 # Adjust stride as needed to reduce the number of quivers
    lon_downsampled = lon[::stride, ::stride]
    lat_downsampled = lat[::stride, ::stride]
    u10_downsampled = u10[::stride, ::stride]
    v10_downsampled = v10[::stride, ::stride]
    
    #quiver function (lon, lat, u, v)
    ax.quiver(lon_downsampled, lat_downsampled, u10_downsampled, v10_downsampled, transform=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_extent(extent)
    ax.set_title(f"{date}")

plt.tight_layout()
#plt.savefig('SSTvectormaps.png')
plt.show()

In [ ]:
#Same code as above except plotting Ekman Transport values
dates = ['2003-01-31','2004-01-31','2005-01-31', '2006-01-31', '2007-01-31', '2008-01-31','2009-01-31', '2010-01-31', '2011-01-31']

longitude = new_wind['longitude'].values
latitude = new_wind['latitude'].values

lon, lat = np.meshgrid(longitude, latitude)

extent = [-90, -80, 23.5, 31]

fig, axes = plt.subplots(nrows = 3, ncols = 3 , figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})


for i, date in enumerate(dates):
    ax = axes.flat[i]
    new_wind.sel(time=date, method='nearest')['EkmTrans'].plot(ax=ax, cmap='seismic', cbar_kwargs={'label': 'Upwelling Index'})
    
    u10 = new_wind['u10'].sel(time=date, method='nearest').values
    v10 = new_wind['v10'].sel(time=date, method='nearest').values
    
    stride = 5 # Adjust stride as needed to reduce the number of quivers
    lon_downsampled = lon[::stride, ::stride]
    lat_downsampled = lat[::stride, ::stride]
    u10_downsampled = u10[::stride, ::stride]
    v10_downsampled = v10[::stride, ::stride]
    
    ax.quiver(lon_downsampled, lat_downsampled, u10_downsampled, v10_downsampled, transform=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_extent(extent)
    ax.set_title(f"{date}")

plt.tight_layout()
#plt.savefig('UIvectormaps.png')
plt.show()